In [2]:
import numpy as np
import pandas as pd
import torch
import csv
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [3]:
# open csv file and read in data
data = pd.read_csv('../Datasets/SectionData.csv')

In [12]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers):
        super(RNN, self).__init__()
        self.model = nn.Sequential(
            nn.RNN(input_size, hidden_size[0], kernel_size),
            # nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(input_size - kernel_size + 1, hidden_size[0]),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.Softmax(dim=1) # MSELoss
        )
    
    def forward(self, x):
        out = self.model(x)
        
        return out

In [13]:
def formatDataX(data):
    temp = torch.tensor(data).float()
    return temp.view(temp.size(0), 1, temp.size(1))

def formatDataY(data):
    return torch.tensor(data).float()

In [14]:
def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = np.array([np.insert(np.zeros(5), label, 1) for label in df.iloc[:, -1].values])
    
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    return formatDataX(X_train), formatDataX(X_val), formatDataX(X_test), formatDataY(Y_train), formatDataY(Y_val), formatDataY(Y_test)

In [15]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [16]:
# size of the input at each time step
input_size = 100
# kernel size
kernel_size = 20
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# dropout probability
drop_prob = 0.2

# instantiate the NN
neuralNet = RNN(input_size, kernel_size, hidden_size, drop_prob)
print(neuralNet)

RNN(
  (model): Sequential(
    (0): RNN(100, 64, num_layers=20)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=81, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): Linear(in_features=32, out_features=6, bias=True)
    (5): Softmax(dim=1)
  )
)


In [17]:
epochs = 100
print_every = 45

X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)

In [18]:
def train(net, X, Y, num_epochs):
    # Mean Squared Error and Adam Optimizer with a learning rate of 0.01
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    
    for epoch in range (num_epochs):
        optimizer.zero_grad()
        out = net(X)
        
        loss = criterion(out, Y)
        
        loss.backward()
        optimizer.step()
        
        print("Epoch {} of {} => Loss {:.4f}\r".format(epoch+1, num_epochs, loss.item()), end="")
        
    return net

In [19]:
trained_NN = train(neuralNet, X_train, Y_train, epochs)

RuntimeError: bad allocation